In [11]:
import os
import sapdi
from sapdi.artifact import Artifact, ArtifactKind, ArtifactFileType
from sapdi.internal.datalake.datalake_storage import DatalakeStorage
from sapdi.scenario.context import Context
import pandas as pd
import pickle
import uuid
from io import BytesIO
os.environ["DI_SDK_RUNTIME_CONTEXT"] = "DI_EXECUTION"
Context.context = None


import uuid
from sapdi.scenario.scenario import Scenario
from sapdi.scenario.context import Context
from sapdi.internal.datalake.datalake_storage import DatalakeStorage
from sapdi.internal.datalake.datalake_client import DataLakeClient
from sapdi.internal.vrep.sharedvolume import SharedVolume
from sapdi.internal.di_client import DIClient

scenario_id = None # To fill if you execute the pipeline modeller in a scenario context
if not scenario_id:
    scenario = Scenario(scenario_id=str(uuid.uuid4()), created_by='me', created_at='2020-01-04-15:08:00')
else:
    scenario = Scenario.get(scenario_id=scenario_id)

Context.context = Context(
    scenario=scenario,
    default_storage=DatalakeStorage(),
    runtime_context_env='DI_EXECUTION',
    shared_volume=SharedVolume()
)

DIClient._instance = None
DataLakeClient._instance = None

class DLPersistor:
    @staticmethod
    def create_dataset_artifact(file_name, file_directory):
        artifact = sapdi.create_artifact(
            artifact_kind=ArtifactKind.DATASET,
            artifact_name=file_name + "-" + str(uuid.uuid4()),
            upload_content=os.path.join(file_directory, file_name),
            description=file_name)
        return artifact

    @staticmethod
    def get_dataframe_from_dataset_artifact(artifact_id):
        artifact = sapdi.get_artifact(artifact_id=artifact_id)
        file_handler = artifact.open_file(file_name=artifact.description)
        content = file_handler.read()
        df = pd.DataFrame([x.split(',') for x in content.decode('utf-8').split('\n')])
        df = pd.DataFrame(df.values[1:], columns=df.iloc[0])
        return df

    @staticmethod
    def create_model_artifact(file_name, file_directory):
        artifact = sapdi.create_artifact(
            artifact_kind=ArtifactKind.MODEL,
            artifact_name=str(uuid.uuid4()),
            upload_content=os.path.join(file_directory, file_name),
            description=file_name)
        return artifact

    @staticmethod
    def get_model_from_artifact(artifact_id):
        artifact = sapdi.get_artifact(artifact_id=artifact_id)
        file_handler = artifact.open_file(file_name=artifact.description)
        content = file_handler.read()
        model = pickle.loads(content)
        return model
        
model1 = pickle.dumps('test_model')
# f = open("Titanic/model1", "wb+")
# f.write(model1)
# f.close()

dl_persistor = DLPersistor()
artifact = dl_persistor.create_model_artifact("modelT", "Titanic")
print(artifact.artifact_id)
art_id = artifact.artifact_id  # '8b283313-3a05-498a-ae20-7a7083bf3320'  # artifact.artifact_id

model = dl_persistor.get_model_from_artifact(art_id)
print(model)



30cc1a55-3179-41b1-90a8-017f61018131
MLPClassifier(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
              beta_2=0.999, early_stopping=False, epsilon=1e-08,
              hidden_layer_sizes=(30, 20, 10), learning_rate='constant',
              learning_rate_init=0.001, max_fun=15000, max_iter=200,
              momentum=0.9, n_iter_no_change=10, nesterovs_momentum=True,
              power_t=0.5, random_state=None, shuffle=True, solver='adam',
              tol=0.0001, validation_fraction=0.1, verbose=False,
              warm_start=False)


In [18]:
model.predict([[1,1,1,0,5,3,3,1,3], [1,1,1,0,5,3,3,1,3], [1,1,1,0,5,3,3,1,3]]).tolist()

[1, 1, 1]